In [38]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

arcpy.env.overwriteOutput = True

# ** Librerías propias


In [39]:
RUTA_BD_CONSOLIDADA_NUBE = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Consolidacion_ES\_con_r_lc_terreno"
RUTA_TERRENOS_GC = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO"

BD_CONSOLIDADA_LOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada"
CAPA_TERRENOS_W_GIS = 'TERRENO_UNIFICADO_w_GIS'
ruta_fc_terrenos_w_gis = os.path.join(BD_CONSOLIDADA_LOCAL,CAPA_TERRENOS_W_GIS)

In [40]:
df_terrenos_gis = pd.DataFrame.spatial.from_featureclass(RUTA_BD_CONSOLIDADA_NUBE)
df_terrenos_gc = pd.DataFrame.spatial.from_featureclass(RUTA_TERRENOS_GC)

columna_creacion_terrenos_gis = {'codigo_anterior':None}

columnas_seleccion_terrenos_gis = ['terreno_codigo',
    'codigo_anterior',
    'created_user',
    'last_edited_user',
    'last_edited_date',
    'globalid',
    'Area_ha_cmt12',
    'SHAPE'
    ]

renombre_columnas_terrenos_gis = {
    'terreno_codigo':'codigo',
    'created_user':'sig_creador',
    'last_edited_user':'sig_ultima_edicion',
    'last_edited_date':'fecha_ultima_edicion',
    'Area_ha_cmt12':'area_ha_cmt12'
    }

creacion_columnas_no_editado = {
    'sig_creador':None,
    'sig_ultima_edicion':None,
    'fecha_ultima_edicion':None,
    'globalid':None,
    }

columnas_normalizacion_terreno_no_editado = ['codigo',
    'codigo_anterior',
    'area_ha_cmt12',
    'SHAPE_x'
    ]

columnas_reorganizacion_no_editado = ['codigo',
    'codigo_anterior',
    'sig_creador',
    'sig_ultima_edicion',
    'fecha_ultima_edicion',
    'globalid',
    'area_ha_cmt12',
    'SHAPE_x'
    ]

In [41]:
df_terrenos_gc

,OBJECTID,codigo,codigo_anterior,area_ha_cmt12,SHAPE
0,1,086060002000000000077000000000,08606000200000077000,20.362073,"{""rings"": [[[4773313.9249, 2723147.2433], [477..."
1,2,086060002000000000001000000000,08606000200000001000,84.724294,"{""rings"": [[[4768164.8455, 2730405.9418], [476..."
2,3,086060002000000000002000000000,08606000200000002000,382.896327,"{""rings"": [[[4771587.4849, 2729015.0273], [477..."
3,4,086060002000000000003000000000,08606000200000003000,31.495322,"{""rings"": [[[4768325.4354, 2729083.0973000005]..."
4,5,086060002000000000004000000000,08606000200000004000,43.230975,"{""rings"": [[[4767838.0119, 2729745.0450999998]..."
...,...,...,...,...,...
39734,40010,080780100000001070007000000000,08078010001070007000,0.010442,"{""rings"": [[[4789783.365, 2751754.2195999995],..."
39735,40011,080780100000001070021000000000,08078010001070021000,0.014832,"{""rings"": [[[4789789.4954, 2751753.0899], [478..."
39736,40012,080780100000000090013000000000,08078010000090013000,0.004797,"{""rings"": [[[4790916.6702, 2752244.5452999994]..."
39737,40013,080780100000000090006000000000,08078010000090006000,0.015991,"{""rings"": [[[4790924.6077, 2752240.383300001],..."


In [42]:
df_merge_terrenos_gc_gis = pd.merge(left=df_terrenos_gc,right=df_terrenos_gis,left_on="codigo",right_on="terreno_codigo",how="left")
print(f"{df_merge_terrenos_gc_gis.shape[0]}")

39743


In [43]:
df_terrenos_no_editados = df_merge_terrenos_gc_gis[df_merge_terrenos_gc_gis['terreno_codigo'].isnull()]
print(f"{df_terrenos_no_editados.shape[0]}")

34658


In [44]:

for llave, valor in columna_creacion_terrenos_gis.items():
    df_terrenos_gis[llave] = valor

df_terrenos_gis = df_terrenos_gis[columnas_seleccion_terrenos_gis]
df_terrenos_gis = df_terrenos_gis.rename(columns=renombre_columnas_terrenos_gis)

In [45]:


columna_renombre_no_editado = {'SHAPE_x':'SHAPE'}

df_terrenos_no_editados = df_terrenos_no_editados[columnas_normalizacion_terreno_no_editado]

df_terrenos_no_editados = df_terrenos_no_editados.copy()

for llave, valor in creacion_columnas_no_editado.items():
    df_terrenos_no_editados[llave] = valor

df_terrenos_no_editados = df_terrenos_no_editados[columnas_reorganizacion_no_editado]

df_terrenos_no_editados = df_terrenos_no_editados.rename(columns=columna_renombre_no_editado)

In [46]:
df_terrenos_unificado = pd.concat([df_terrenos_gis, df_terrenos_no_editados])

df_terrenos_unificado.spatial.to_featureclass(location = ruta_fc_terrenos_w_gis)
print(f"Se crea la capa {CAPA_TERRENOS_W_GIS} en BD \n {BD_CONSOLIDADA_LOCAL}")

Se crea la capa TERRENO_UNIFICADO_w_GIS en BD 
 C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada


In [48]:
arcpy.env.workspace = BD_CONSOLIDADA_LOCAL


for capa in arcpy.ListFeatureClasses(wild_card='TERRENO_UNIFICADO_w_GIS'):
    for campo in arcpy.ListFields(dataset=capa, wild_card='area_ha_cmt12'):
        arcpy.management.CalculateGeometryAttributes(in_features = capa, geometry_property = [[campo.name,'AREA']], area_unit='HECTARES')
        print(f"Se actualiza el campo de hectareaje")

Se actualiza el campo de hectareaje
